In [1]:
%load_ext autoreload


In [2]:
# Add path to main folder
import sys
sys.path.append('../')
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from cytoImage import image
import cv2
import pandas as pd
import itertools as iter
import copy




In [5]:
# Import image
sys.path.append('../sourceImages')
cell_img = image.Image.from_file('../sourceImages/RK1.jpg', 'Original Image')
cell_img.im_show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Threshold the image
cell_img_thresh1 = cell_img.bin_threshold(120,255)
cell_img_thresh1.im_show()
cell_img_thresh1_inv = cell_img_thresh1.bin_invert()
cell_img_thresh1_inv.im_show()

kernel = np.zeros((31,31), np.uint8)
kernel = cv2.circle(kernel, (15,15), 15, color=(1), thickness=2 )

cell_img_eroded = cell_img_thresh1.bin_erode(kernel, 1)
cell_img_eroded.im_show()
# img2 = cell_img_thresh1_inv.image
# img2[img2 > 0] = 1
# cell_img_eroded = cv2.erode(img2, kernel, iterations=1)








Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Create algorithm for area detection
img_array = cell_img_thresh1.image
img_array = img_array   # Transpose to set x&y to image coords
img_array[img_array > 0] = 1        # Set 255 values to 1's

def get_length(img):
    # Get Indices of gap lengths
    indices = []
    for x in range(img.shape[0]-1):
        indices.append([])
        y = 0
        while (y < img.shape[1]-1):
            if ((img[x,y]==0) and (img[x,y]==img[x,y+1])):
                acc = 1; ind_start = [x,y]; ind_end = [x,y]
                while ((y != img.shape[1]-1) and (img[x,y] == img[x,y+1])):
                    acc += 1
                    y   += 1
                ind_end[1] += acc
                y += 1
                indices[x].append((ind_start, ind_end))
            y += 1
    # print(indices)

    # Find the longest stretch of wound in each row
    longest_arr = []
    for i in range(len(indices)):
        longest = None
        for j in range(len(indices[i])):
            if not longest:
                longest = indices[i][j]
            elif ( (indices[i][j][1][1]-indices[i][j][0][1]) > longest[1][1] - longest[0][1]):
                longest = indices[i][j]
            else:
                continue
        longest_arr.append(longest)
    print(longest_arr[650])

get_length(img_array)





([650, 501], [650, 1441])


In [6]:
img = np.array([[1,1,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0],
                [1,0,0,0,1,1,1,0,0,0,0,0,1,1,1,1,0,0],
                [1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0]])


# Get Indices of gap lengths
indices = []
for x in range(img.shape[0]):
    indices.append([])
    y = 0
    while (y < img.shape[1]):
        # if (y != img.shape[1]-1):
        if ((img[x,y]==0) and (img[x,y]==img[x,y+1])):
            acc = 1; ind_start = [x,y]; ind_end = [x,y]
            while ( (y != img.shape[1]-1) and (img[x,y] == img[x,y+1]) ):
                acc += 1
                y   += 1
            ind_end[1] += acc
            y += 1
            indices[x].append((ind_start, ind_end))
        y += 1
print(indices)

# Find the longest stretch of wound in each row
longest_arr = []
for i in range(len(indices)):
    longest = None
    for j in range(len(indices[i])):
        if not longest:
            longest = indices[i][j]
        elif ( (indices[i][j][1][1]-indices[i][j][0][1]) > longest[1][1] - longest[0][1]):
            longest = indices[i][j]
        else:
            continue
    longest_arr.append(longest)
print(longest_arr)





[[([0, 2], [0, 5]), ([0, 7], [0, 13]), ([0, 16], [0, 18])], [([1, 1], [1, 4]), ([1, 7], [1, 12]), ([1, 16], [1, 18])], [([2, 3], [2, 6]), ([2, 16], [2, 18])]]
[([0, 7], [0, 13]), ([1, 7], [1, 12]), ([2, 3], [2, 6])]


In [3]:
# Kernel Testing
kernel = np.zeros((5,5), np.uint8)
kernel = cv2.circle(kernel, (2,2), 2, color=(1), thickness=1 )

img = np.zeros((20,20), np.uint8)
img[5,6] = 255
img[7,9] = 255
img[11,3] = 255

img = image.Image(img, 'Small Test Image')

img_invert = img.bin_invert()
img_invert.im_show()
img_morphed = img_invert.bin_erode(kernel,1)
img_morphed.im_show()





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …